In [1]:
## Libraries
import pandas as pd
import numpy as np
import scipy, scipy.stats

### 2. Write a function ‘exists’ which takes a variable symbol v and returns whether v is defined.

In [36]:
try:
    var
except NameError:
    print('v is not defined')
else:
    print("exists")

v is not defined


### 3. Create a function to compute N layer of a Pascal Triangle. 

In [37]:
def pascal_triangle(n):
    pascal = [[1]*(i+1) for i in range(n)] # set the initial layer
    
    # O(n^2)
    for i in range(n):
        for j in range(1,i):
            pascal[i][j] = pascal[i-1][j-1] + pascal[i-1][j]
    return pascal

In [38]:
# test cases for pascal triangle
pascal_triangle(10)
# pascal_triangle(20)

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1]]

### 4. Assume have the following portfolio as of 2016/01/01:
    AAPL.O 15%
    IBM.N 20%
    GOOG.O 20%
    BP.N 15%
    XOM.N 10%
    COST.O 15%
    GS.N 5%

a. Using historical daily returns (Yahoo/Google Finance or any other market data
source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

b. Using expected mean, covariance matrix and parametric method, calculate VaR95%
and CVaR95%

c. Assume you can change weights, allow shorting but no leverage (i.e. sum of weights
equal 100%), and rebalance monthly. What is the optimal portfolio holding by end of
each month till end of 2016
Notes: If you have other assumption(s) please state clearly

In [2]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['date'] = pd.to_datetime(portfolio['date'], format='%d/%m/%Y')

# calculate returns
portfolio.iloc[:, 1:8] = portfolio.iloc[:, 1:8].pct_change(1)

# portfolio weights
weights = np.array([.15, .2, .2, .15, .1, .15, .05])

#### a. Using historical daily returns (Yahoo/Google Finance or any other market data source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31

In [3]:
# returns of portfolio for historical simulation of VaR
returns = portfolio.copy()
returns.set_index('date', inplace=True)
returns.dropna(axis=0, inplace=True)

In [4]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2016-01-05 to 2016-12-30
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   aapl    251 non-null    float64
 1   ibm     251 non-null    float64
 2   goog    251 non-null    float64
 3   bp      251 non-null    float64
 4   xom     251 non-null    float64
 5   cost    251 non-null    float64
 6   gs      251 non-null    float64
dtypes: float64(7)
memory usage: 15.7 KB


In [12]:
# find the portfolio returns with weights
portfolio_returns = (weights * returns.values).sum(axis=1)

Equation for CVAR:

$$CVaR(\alpha) = \frac{1}{1-\alpha}\mathbb{E}\int_{VaR(\alpha)}^x xf(x)dx $$

In [50]:
historic_var = round(np.percentile(pnl, 5, interpolation="lower"), 3)

# using scipy norm to calculate cvar
c_var_95 =  (1/(1 - 0.95))*scipy.stats.norm.expect(lambda x: x, lb = historic_var)

print('The simple historical VaR95% is {}'.format(historic_var))
print('The CVaR95% is {}'.format(c_var_95))

The simple historical VaR95% is -0.015
The CVaR95% is 7.977948038386972


#### b. Using expected mean, covariance matrix and parametric method, calculate VaR95% and CVaR95%

In [26]:
portfolio.head()

,date,aapl,ibm,goog,bp,xom,cost,gs
0,2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-05,-0.025057,-0.000736,0.000998,-0.004185,0.008521,0.002445,-0.017218
2,2016-01-06,-0.019470,-0.005006,0.001401,-0.020369,-0.008321,-0.009254,-0.024413
3,2016-01-07,-0.042494,-0.017090,-0.023170,-0.028713,-0.016006,-0.022909,-0.030735
4,2016-01-08,0.005392,-0.009258,-0.016410,-0.017669,-0.020202,-0.017504,-0.004131


In [6]:
# cov matrix
cov_mat = portfolio.set_index('date').cov()

# avg returns
avg_ret = portfolio.set_index('date').mean()

portfolio_mean = avg_ret.dot(weights)

# portfolio std dev
portfolio_sd = np.sqrt(weights.T.dot(cov_mat).dot(weights))

# mean of investment


In [7]:
portfolio_mean

0.0005788264849869774

In [13]:
# function to return stats of portfolio
def stats(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    df.set_index('date', inplace=True)

    # calculate returns
    returns = df.pct_change(1)
    returns.dropna(axis=0, inplace=True)
    cov_mat = returns.cov()
    return returns, cov_mat

In [21]:
def historical_var(returns, alpha=5):
    return np.percentile(returns, alpha)

In [19]:
historic_var

-0.014251755882693262

In [20]:
portfolio_returns.aggregate(historic_var, 5)

AttributeError: 'numpy.ndarray' object has no attribute 'aggregate'